In [ ]:
## DYNAMIC MOTION DETECTION.

import numpy as np
import cv2

# initializing the web cam
vid = cv2.VideoCapture(0)
succ1,frame1 = vid.read()
if not succ1:
    print("Failed to start video")
    exit()

# creating the named Window.
cv2.namedWindow('Dynamic Motion Detection',cv2.WINDOW_FREERATIO)

## Initializing of Global variables.
xi, yi, draw = None, None, False
rois = []
preview_gray = None
preview_rect = None ## to show live preview of rectangle

##defining mouse events fuction.
def draw_roi(event,x,y,flags,user):
    global xi, yi, draw, preview_rect
    if event == cv2.EVENT_LBUTTONDOWN:
        xi, yi = x, y
        draw = True
        
    elif event == cv2.EVENT_MOUSEMOVE and draw == True:
        preview_rect = (xi,yi,x,y) ## live caputing the cordinates and updation untill release the click
        
    elif event == cv2.EVENT_LBUTTONUP:
        draw = False
        preview_rect = None
        rois.append((min(xi,x),min(yi,y),max(xi,x),max(yi,y))) ## first clicked and released cordinates are recorded.
        print(f"📦 ROI added : {rois[-1]}")
        
    elif event == cv2.EVENT_RBUTTONDOWN:
        for i,(x1,y1,x2,y2) in enumerate(rois): ## iteration inside the list to check the right click happened in which roi.
            if x1 <= x <= x2 and y1 <= y <= y2: ## if right click inside the roi region fo any roi, then roi is poped out.
                removed = rois.pop(i)
                print(f"🗃️ ROI removed : {removed}")
        

## creation of mouse callback.
cv2.setMouseCallback('Dynamic Motion Detection',draw_roi)

## instructions.
print('The web cam is started.')
print("left click, drag and release ----> to create [ Region of Intrest(ROI) ], Motion Detection starts within the ROI.")
print("Right click inside the ROI ----> to delete ROI")
print("press ' q ' or ' Q ' to terminate")

## initial frame is none
prev_gray = None

## --- Main Loop ---
while True:
    ## continuesly reads frames from camara.
    succ,frame = vid.read()
    if not succ:
        print("failed to start video")
        break

    ## convertion to grayscale to speed up the process.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    ## if the frame is first skipping the detection, as we need previous frame for detection.
    if prev_gray is None:
        prev_gray = gray
        continue

    ## looping through all the regions of previous and current frames.
    for (x1,y1,x2,y2) in rois:
        pre_roi = prev_gray[y1:y2, x1:x2]
        curr_roi = gray[y1:y2, x1:x2]

        ## skipping if the (x1,y1) == (x2,y2) ---> '0' pixels ( no ROI)
        if pre_roi.size == 0 or curr_roi.size == 0:
            continue

        ## checking for change between previous and current ROI's
        diff = cv2.absdiff(pre_roi,curr_roi)
        motion_level = np.sum(diff)/255

        color = (0, 255, 0) # initially green
        # ROI change more than 1000, reflects in motion detected.
        if motion_level > 1500:
            color = (0, 0, 255) # if motion detects 'red'
            cv2.putText(frame, 'MOTION DETECTED', (x1, max(20, y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        ## drawing the ROI with color.
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    ## live preview of drawing of rectangle with yellow color.
    if preview_rect is not None: 
        x1,y1,x2,y2 = preview_rect # preview_rect captured then starting and ending cordinates from MouseCallback.
        ## ensuring the user may draw from top-left to bottom-right or reverse.
        x_min, y_min = min(x1, x2), min(y1, y2)
        x_max, y_max = max(x1, x2), max(y1, y2)
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,255,255), 2) ## preview rectangle

    ## show
    cv2.imshow('Dynamic Motion Detection',frame)

    ## frames updation
    prev_gray = gray

    ## breaking the loop
    if cv2.waitKey(1) in [ord('q'), ord('Q')]:
        break

## clearing all the windows.
cv2.destroyAllWindows()
            

The web cam is started.
left click, drag and release ----> to create [ Region of Intrest(ROI) ], Motion Detection starts within the ROI.
Right click inside the ROI ----> to delete ROI
press ' q ' or ' Q ' to terminate
📦 ROI added : (142, 135, 662, 573)
📦 ROI added : (1319, 169, 1750, 664)
📦 ROI added : (628, 137, 1210, 834)
📦 ROI added : (408, 329, 1573, 1055)
🗃️ ROI removed : (628, 137, 1210, 834)
🗃️ ROI removed : (408, 329, 1573, 1055)
🗃️ ROI removed : (142, 135, 662, 573)
🗃️ ROI removed : (1319, 169, 1750, 664)
📦 ROI added : (1289, 183, 1831, 626)
